<a href="https://colab.research.google.com/github/carlosmatherson/SULI-Project/blob/main/qm9_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spektral

In [ ]:
# import
import numpy as np

from keras.layers import Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import BatchLoader
from spektral.datasets import QM9
from spektral.layers import ECCConv, GlobalSumPool, GraphMasking

In [ ]:
# config
learning_rate = 1e-3  # Learning rate
epochs = 10  # Number of training epochs
batch_size = 32  # Batch size

In [ ]:
# load data
dataset = QM9(amount=1000)  # Set amount=None to train on whole dataset
print(dataset)

Loading QM9 dataset.
Reading SDF


100%|██████████████████████████████████████| 1000/1000 [00:01<00:00, 713.08it/s]


QM9(n_graphs=1000)


In [ ]:
# parameters
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target
print(dataset[3])

Graph(n_nodes=4, n_node_features=10, n_edge_features=4, n_labels=19)


In [ ]:
# train/test split
idxs = np.random.permutation(len(dataset))
split = int(0.9 * len(dataset))
idx_tr, idx_te = np.split(idxs, [split])
dataset_tr, dataset_te = dataset[idx_tr], dataset[idx_te]
print(dataset)

QM9(n_graphs=1000)


In [ ]:
# build model
class Net(Model):
    def __init__(self):
        super().__init__()
        self.masking = GraphMasking()
        self.conv1 = ECCConv(32, activation="relu")
        self.conv2 = ECCConv(32, activation="relu")
        self.global_pool = GlobalSumPool()
        self.dense = Dense(n_out)

    def call(self, inputs):
        x, a, e = inputs
        x = self.masking(x)
        x = self.conv1([x, a, e])
        x = self.conv2([x, a, e])
        output = self.global_pool(x)
        output = self.dense(output)

        return output

model = Net()
optimizer = Adam(learning_rate)
model.compile(optimizer=optimizer, loss="mse")

In [ ]:
# fit model
loader_tr = BatchLoader(dataset_tr, batch_size=batch_size, mask=True)
model.fit(loader_tr.load(), steps_per_epoch=loader_tr.steps_per_epoch, epochs=epochs)

/usr/local/lib/python3.7/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'QM9' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)
/usr/local/lib/python3.7/dist-packages/spektral/layers/convolutional/conv.py:94: UserWarning: The adjacency matrix of dtype <dtype: 'int64'> is incompatible with the dtype of the node features <dtype: 'float32'> and has been automatically cast to <dtype: 'float32'>.
  f"The adjacency matrix of dtype {a.dtype} is incompatible with the dtype "


Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: The adjacency matrix of dtype <dtype: 'int64'> is incompatible with the dtype of the node features <dtype: 'float32'> and has been automatically cast to <dtype: 'float32'>.
  return py_builtins.overload_of(f)(*args)


29/29 [==============================] - 9s 248ms/step - loss: 13950668.0000
Epoch 2/10
29/29 [==============================] - 5s 170ms/step - loss: 15703727.0000
Epoch 3/10
29/29 [==============================] - 5s 170ms/step - loss: 16583224.0000
Epoch 4/10
29/29 [==============================] - 5s 162ms/step - loss: 16563199.0000
Epoch 5/10
29/29 [==============================] - 5s 159ms/step - loss: 16321137.0000
Epoch 6/10
29/29 [==============================] - 4s 153ms/step - loss: 14943460.0000
Epoch 7/10
29/29 [==============================] - 4s 153ms/step - loss: 14902682.0000
Epoch 8/10
29/29 [==============================] - 4s 153ms/step - loss: 14359013.0000
Epoch 9/10
29/29 [==============================] - 5s 157ms/step - loss: 17506822.0000
Epoch 10/10
29/29 [==============================] - 4s 153ms/step - loss: 17110208.0000


In [ ]:
# evaluate model
print("Testing model")
loader_te = BatchLoader(dataset_te, batch_size=batch_size, mask=True)
loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done. Test loss: {}".format(loss))